In [9]:
import os
from pydantic import BaseModel, Field

class LLMParameters(BaseModel):
    """LLM Parameters model."""
    
    ############################ 
    # Load from .env by default
    api_key: str = Field(
        description="The API key to use for the LLM service.",
        default_factory=lambda: os.getenv("OPENAI_API_KEY"), 
    )
    
    api_base: str | None = Field(
        description="The base URL for the LLM API.",
        default_factory=lambda: os.getenv("OPENAI_API_BASE"), 
    )
    
    api_version: str | None = Field(
        description="The version of the LLM API to use.",
        default_factory=lambda: os.getenv("OPEN_AI_VERSION"), 
    )
    
    deployment_name: str | None = Field(
        description="The deployment name to use for the LLM service.",
        default_factory=lambda: os.getenv("GPT_DEPLOYMENT_NAME"),  
    )
    ############################
    
    temperature: float | None = Field(
        description="The temperature to use for token generation.",
        default=0,
    )



In [10]:
from devtools import pformat

from pydantic import BaseModel, Field

from llm.config.enum import LLMType
from llm.config.llm_parameters import LLMParameters

class LLMConfig(BaseModel):
    
    def __repr__(self) -> str:
        """Get a string representation."""
        return pformat(self, highlight=False)
    
    type: LLMType = Field(
        description='The type of LLM model to sue', default=LLMType.AzureOpenAIChat
    )
    
    llm: LLMParameters = Field(
        description="The LLM configuration to use.", default=LLMParameters()
    )

In [11]:
config = LLMConfig()

In [4]:
config.llm.api_base

'https://yfyai-aoai-instance.openai.azure.com/'

In [5]:
class TestC:
    def __init__(self, config: LLMConfig):
        self.config = config

    def print_config(self):
        return print(self.config)

In [12]:
testc = TestC(config=LLMConfig())

In [15]:
import yaml

In [16]:
with open('configs.yaml', 'r') as f:
    config_data = yaml.safe_load(f)

In [18]:
config_data['Training']

{'usegpu': True,
 'seed': 42,
 'batch_size': 300,
 'lr': '1e-3',
 'n_hidden': [16, 32, 32, 16],
 'epochs': 100,
 'N_fold': 5,
 'optimizer': 'Adam',
 'weight_decay': '1e-4'}

In [57]:
def load_config_from_file(file_path: str) -> LLMConfig:
    """Load LLM configuration from a YAML file."""
    with open(file_path, "r") as file:
        config_data = yaml.safe_load(file)
    return LLMConfig(**config_data.get("LLM", {}))

In [58]:
llm_config = load_config_from_file('configs.yaml')

In [42]:
llm_config

LLMConfig(
    type='azure_openai_chat',
    llm=LLMParameters(
        api_key='c60459f1ee6b4182b16c146727a3112a',
        api_base='https://yfyai-aoai-instance.openai.azure.com/',
        api_version='2024-04-01-preview',
        deployment_name='gpt4o',
        temperature=0,
    ),
)

In [ ]:
from enum import Enum
from predictor.config.ml_configs import NNhyperparameters
import yaml

class ConfigType(Enum):

    LLM = 'llm'
    Model_NN = 'Model_NN'

    def __repr__(self) -> str:
        return f"'{self.value.lower()}'"


def normalize_keys(d):
    if isinstance(d, dict):
        return {k.lower(): normalize_keys(v) for k, v in d.items()}
    return d

class YamlLoader:
    def __init__(self, file_path):
        self.config_data = self.load_config_from_file(file_path)
        print(self.config_data)
        
    def load_config_from_file(self, file_path: str):
        with open(file_path, "r") as file:
            config_data = yaml.safe_load(file)
        return normalize_keys(config_data)
    
    def get_llm_config(self):
        return LLMConfig(**self.config_data.get(ConfigType.LLM, {}))
    
    def get_model_nn_config(self):
        return NNhyperparameters(**self.config_data.get(ConfigType.Model_NN, {}))
    
    

In [19]:
yaml_config = YamlLoader('configs.yaml')

{'futuredataname': 'Bleached Softwood Kraft Pulp Futures Historical Data.csv', 'llm': {'llm_type': 'azure_openai_chat', 'temperature': 0}, 'model_nn': {'seed': 42, 'batch_size': 300, 'lr': '1e-3', 'n_hidden': [16, 32, 32, 16], 'epochs': 100, 'n_fold': 5, 'optimizer': 'Adam', 'weight_decay': '1e-4', 'accelerator': 'gpu'}}


In [15]:
yaml_config.get_llm_config()

LLMConfig(
    type='azure_openai_chat',
    llm=LLMParameters(
        api_key='c60459f1ee6b4182b16c146727a3112a',
        api_base='https://yfyai-aoai-instance.openai.azure.com/',
        api_version='2024-04-01-preview',
        deployment_name='gpt4o',
        temperature=0,
    ),
)

In [21]:
yaml_config.config_data['futuredataname']

'Bleached Softwood Kraft Pulp Futures Historical Data.csv'